### 설치 & 라이브러리


In [3]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 23.3 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu ja

In [4]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.26.1


In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options

In [6]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### url 가져오기


In [58]:
import re
import numpy as np

#selenium driver 로드
driver = webdriver.Chrome(options=chrome_options)
base_url = "https://www.jobploy.kr/ko/recruit?page="

#URL 정보를 받을 리스트
url_list = []

# 한 번에 실행하면 너무 오래 걸려서 10개 단위로 나눠서 실행
max_pages = 110

#링크 전달
for page in range(101, max_pages + 1):
  url = f"{base_url}{page}"
  driver.get(url)

  #CSS_SELECTOR를 활용해서 section 태그를 가져오기
  sections = driver.find_element(By.CSS_SELECTOR, "#app > div.content_wrap > section > div.content_group > div > div.right_content > div > div > div.content> div")
  a_tags = sections.find_elements(By.TAG_NAME, "a")
  hrefs = [a.get_attribute("href") for a in a_tags if a.get_attribute("href") and "page=" not in a.get_attribute("href")]
  url_list.extend(hrefs)

#driver 종료
driver.quit()

In [105]:
# url 확인
url_list

['https://www.jobploy.kr/ko/recruit/RS0000058520',
 'https://www.jobploy.kr/ko/recruit/RS0000058519',
 'https://www.jobploy.kr/ko/recruit/RS0000058518',
 'https://www.jobploy.kr/ko/recruit/RS0000058517',
 'https://www.jobploy.kr/ko/recruit/RS0000058516',
 'https://www.jobploy.kr/ko/recruit/RS0000058515',
 'https://www.jobploy.kr/ko/recruit/RS0000058514',
 'https://www.jobploy.kr/ko/recruit/RS0000058513',
 'https://www.jobploy.kr/ko/recruit/RS0000058512',
 'https://www.jobploy.kr/ko/recruit/RS0000058511',
 'https://www.jobploy.kr/ko/recruit/RS0000058510',
 'https://www.jobploy.kr/ko/recruit/RS0000058509',
 'https://www.jobploy.kr/ko/recruit/RS0000058508',
 'https://www.jobploy.kr/ko/recruit/RS0000058507',
 'https://www.jobploy.kr/ko/recruit/RS0000058506',
 'https://www.jobploy.kr/ko/recruit/RS0000058505',
 'https://www.jobploy.kr/ko/recruit/RS0000058504',
 'https://www.jobploy.kr/ko/recruit/RS0000058503',
 'https://www.jobploy.kr/ko/recruit/RS0000058502',
 'https://www.jobploy.kr/ko/rec

### 크롤링 함수

In [53]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

def scrape_job_info(url):

  # Selenium driver 로드
  driver = webdriver.Chrome(options=chrome_options)

  # 링크 전달
  driver.get(url)

  # CSS_SELECTOR를 활용해서 div 태그를 가져오기
  div = driver.find_element(By.CSS_SELECTOR, "#app > div.content_wrap")

  # 정보 저장을 위한 딕셔너리 초기화
  job_info = {}

  # 페이지가 완전히 로드될 때까지 기다기
  WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div.content_wrap"))
  )

   # 회사 이름 가져오기
  try:
    company_name = div.find_element(By.CSS_SELECTOR, "section.title_top > div > div.top_line.d-flex.justify-content-between.align-items-end > h4")
    job_info['회사 이름'] = company_name.text
  except NoSuchElementException:
    job_info['회사 이름'] = ""

  # 공고 제목 가져오기
  try:
    title = div.find_element(By.CSS_SELECTOR, "section.title_top > div > h3")
    job_info['제목'] = title.text
  except NoSuchElementException:
    job_info['제목'] = ""

  # 등록 날짜 가져오기
  try:
    start_date = div.find_element(By.CSS_SELECTOR, "section.title_top > div > div.top_line.d-flex.justify-content-between.align-items-end > small")
    start_date_text = start_date.text
    job_info['등록 날짜'] = start_date_text.replace("등록 : ", "").strip()
  except NoSuchElementException:
    job_info['등록 날짜'] = ""

  # 지역 가져오기
  try:
    region = div.find_element(By.CSS_SELECTOR, "section.title_top > div > div.recruit-tags > span:nth-child(1)")
    job_info['지역'] = region.text
  except NoSuchElementException:
    job_info['지역'] = ""

  # 급여 가져오기
  try:
    salary = div.find_element(By.CSS_SELECTOR, "section.title_top > div > div.recruit-tags > span:nth-child(2)")
    job_info['급여'] = salary.text
  except NoSuchElementException:
    job_info['급여'] = ""

  # 직무 가져오기
  try:
    job = div.find_element(By.CSS_SELECTOR, "section.title_top > div > div.recruit-tags > span:nth-child(3)")
    job_text = job.text
    # 직무 텍스트를 ">"를 기준으로 나누기
    if ">" in job_text:
      job_info['직무 분야'], job_info['직무 세부'] = job_text.split(">", 1)
      job_info['직무 분야'] = job_info['직무 분야'].strip()
      job_info['직무 세부'] = job_info['직무 세부'].strip()
    else:
      job_info['직무 분야'] = job_text.strip()
      job_info['직무 세부'] = ""  # '>'가 없을 경우 세부 직무는 빈 값으로 설정
  except NoSuchElementException:
    job_info['직무 분야'] = ""
    job_info['직무 세부'] = ""

  # 채용 마감일 가져오기
  try:
    end_date = div.find_element(By.CSS_SELECTOR, "section.title_top > div > div.recruit-tags > span:nth-child(4)")
    end_date_text = end_date.text
    cleaned_date = end_date_text.replace("채용마감 : ", "").strip()
    job_info['채용 마감일'] = cleaned_date
  except NoSuchElementException:
    job_info['채용 마감일'] = ""

  # 상세 근무 요강
  try:
    details = div.find_element(By.CSS_SELECTOR, "section.container-fluid.pt-4\\.5 > div > div:nth-child(1) > div > div > div > div")
    job_info['상세 근무 요강'] = details.text
  except NoSuchElementException:
    job_info['상세 근무 요강'] = ""

  # driver 종료
  driver.quit()

  return job_info

In [ ]:
# 함수 확인
job_info = scrape_job_info(url[0])
job_info

### 크롤링하기

In [12]:
all_job_info = []
for url in url_list:
  job_info = scrape_job_info(url)
  all_job_info.append(job_info)

In [59]:
for url in url_list:
  job_info = scrape_job_info(url)
  all_job_info.append(job_info)

### 엑셀 파일로 만들기

In [60]:
df = pd.DataFrame(all_job_info)

# 결측치, 중복값 제거
cleaned_df = df[~(df == "").any(axis=1)]
deduplicated_df = cleaned_df.drop_duplicates()
final_df = deduplicated_df.reset_index(drop=True)

final_df

In [ ]:
final_df.to_excel("job_info.xlsx")

### 상세 정보 크롤링 (2차 구현, 시도하다 실패한 코드)

In [16]:
import re

# Selenium driver 로드
driver = webdriver.Chrome(options=chrome_options)

#링크 전달
driver.get("https://www.jobploy.kr/ko/recruit/R0000058362")

#CSS_SELECTOR를 활용해서 div 태그를 가져오기
div = driver.find_element(By.CSS_SELECTOR, "#app > div.content_wrap")

# 상세 근무 요강
details = div.find_element(By.CSS_SELECTOR, "section.container-fluid.pt-4\.5 > div > div:nth-child(1) > div > div > div > div")
job_info["상세 근무 요강"] = details.text

# 모집 조건
recruit_conditions = div.find_element(By.CSS_SELECTOR, "section.container-fluid.pt-4\.5 > div > div:nth-child(1) > div > div > div > div")


sections = {
  "상세 근무 요강" : "div:nth-child(1)",
  "모집 조건": "div:nth-child(2)",
  "근무 조건": "div:nth-child(3)",
  "복리후생": "div:nth-child(4)",
  "근무지": "div:nth-child(5)",
  "회사 정보": "div:nth-child(6)"
}

for section_name, selector in sections.items():
    # 각 섹션에 대해 딕셔너리 생성
    section_dict = {}

    # 해당 섹션을 가져오기
    try:
        section = driver.find_element(By.CSS_SELECTOR, f"#app > div.content_wrap > section.container-fluid.pt-4\\.5 > div > {selector}")
        rows = section.find_elements(By.CLASS_NAME, "col-12")

        # 각 속성명과 값을 딕셔너리에 저장
        for row in rows:
            try:
                key = row.find_element(By.CLASS_NAME, "col_title.col-form-label").text.strip()
                value = row.find_element(By.CLASS_NAME, "col").text.strip()
                section_dict[key] = value
            except Exception as e:
                print(f"Error in row: {e}")

        # 전체 정보 딕셔너리에 섹션 추가
        job_info[section_name] = section_dict

    except Exception as e:
        print(f"Error in section {section_name}: {e}")

# 결과 출력
print(job_info)

# driver 종료
driver.quit()

Error in row: Message: no such element: Unable to locate element: {"method":"css selector","selector":".col_title.col-form-label"}
  (Session info: chrome=131.0.6778.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x57255b0064ca <unknown>
#1 0x57255ab19620 <unknown>
#2 0x57255ab68306 <unknown>
#3 0x57255ab685a1 <unknown>
#4 0x57255ab5ca46 <unknown>
#5 0x57255ab8c39d <unknown>
#6 0x57255ab5c938 <unknown>
#7 0x57255ab8c53e <unknown>
#8 0x57255abaade0 <unknown>
#9 0x57255ab8c113 <unknown>
#10 0x57255ab5abe0 <unknown>
#11 0x57255ab5bbbe <unknown>
#12 0x57255afd2e4b <unknown>
#13 0x57255afd6de2 <unknown>
#14 0x57255afbfd2c <unknown>
#15 0x57255afd7957 <unknown>
#16 0x57255afa54bf <unknown>
#17 0x57255aff5348 <unknown>
#18 0x57255aff5510 <unknown>
#19 0x57255b005346 <unknown>
#20 0x7a2aa9f01ac3 <unknown>

Error in row: Message: no such element: Unable to locate element: {"meth

In [64]:
# Selenium driver 로드
driver = webdriver.Chrome(options=chrome_options)

# 링크 전달
driver.get('https://www.jobploy.kr/ko/recruit/RS0000058509')

# # CSS_SELECTOR를 활용해서 div 태그를 가져오기
# div = driver.find_element(By.CSS_SELECTOR, "#app > div.content_wrap")

# address = div.find_element(By.CSS_SELECTOR, "#app > div.content_wrap > section.container-fluid.pt-4\.5.top > div > div:nth-child(5) > div.recruit-field-table > div.row > div > div > div")
# print(address.text)

# # 근무지 주소 네이버 지도 링크
# # '길 찾기' 링크 요소 찾기
# map_link_element = div.find_element(By.CSS_SELECTOR, "section.container-fluid.pt-4\.5.top > div > div:nth-child(5) > div.recruit-field-table > div.row > div > a")
# # href 속성 값 가져오기
# map_link = map_link_element.get_attribute("href")
# print(map_link)

# # driver 종료
# driver.quit()


# 일정 시간 기다리기 (예: 5초)
time.sleep(5)

try:
  # 근무지 주소 네이버 지도 링크
  div = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div.content_wrap"))
  )
  map_link_element = WebDriverWait(div, 10).until(
    EC.presence_of_element_located(
      (By.XPATH, "//section[contains(@class, 'container-fluid')]/div/div[5]//a[contains(@href, 'https://map.naver.com')]")
    )
  )
  # href 속성 값 가져오기
  map_link = map_link_element.get_attribute("href")
  # 네이버 지도 링크 저장
  job_info['네이버 지도 링크'] = map_link

except Exception as e:
    print(f"Error: {e}")

finally:
    driver.quit()